<a href="https://colab.research.google.com/github/GaneshiUmayangana/Deep-learning-Mini-Project-03/blob/main/Deep_Learning_Mini_Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#English to Sinhala Transalation with Transforms



In [72]:
from IPython.display import Image

In [73]:
Image(url ='https://www.tensorflow.org/images/tutorials/transformer/apply_the_transformer_to_machine_translation.gif')

#Necessary Library Imports

#Prepare the Data

In [ ]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

#Mount the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Read the data file

In [ ]:
text_file = "/content/drive/My Drive/Colab_Data_Files/sin.txt"
with open(text_file) as f:
  lines = f.read().split("\n")[:-1]
i = 0
for line in lines:
  print(line)
  i = i + 1
  if(i==20):
    break

Go.	යන්න.
Go.	යන්න.
Go.	යන්න.
Go.	යන්න.
Hi.	ආයුබෝවන්.
Run!	දුවන්න!
Run.	දුවන්න.
Who?	කවුද?
Wow!	වාව්!
Fire!	ගිනි!
Fire!	ගිනි!
Fire!	ගිනි!
Help!	උදව්!
Help!	උදව්!
Help!	උදව්!
Jump!	පනින්න!
Jump.	පනින්න.
Stop!	නවත්වන්න!
Stop!	නවත්වන්න!
Stop!	නවත්වන්න!


In [ ]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

Tom promised.	ටොම් පොරොන්දු විය.
Tom promised.	ටොම් පොරොන්දු විය.
Tom ran away.	ටොම් පලා ගියේය.
Tom relented.	ටොම් පසුබට විය.
Tom relented.	ටොම් පසුබට විය.
Tom relented.	ටොම් අනුකම්පා කළා.
Tom resigned.	තෝමස් ඉල්ලා අස්විය.
Tom saw Mary.	ටොම් මේරිව දැක්කා.
Tom screamed.	ටොම් කෑගැසුවා.
Tom shot her.	ටොමස් ඔහුට වෙඩි තැබුවේය.


#Split the English and Sinhala translation pairs

In [50]:
text_pairs = []

for line in lines:
    parts = line.split("\t")

    if len(parts) == 2:
        english, sinhala = parts
        sinhala = "[start]" + sinhala + "[end]"
        text_pairs.append((english, sinhala))
    else:

        print("Skipping line:", line)

Skipping line: 


In [51]:
for i in range(3):
  print(random.choice(text_pairs))

('I mean it.', '[start]මම බැරෑරුම්.[end]')
('Keep running.', '[start]දිගටම දුවන්න.[end]')
("It's here.", '[start]මෙතන.[end]')


#Randomize the data

In [52]:
import random
random.shuffle(text_pairs)

#Splitting the data into training, validation and testing

In [53]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))

Total sentences: 3500
Training set size: 2450
Validation set size: 525
Testing set size: 525


In [54]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

3500

#Removing Punctuation

In [55]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [56]:
f"{3+5}"


'8'

#vectorizing the English and Sinhala text pairs

In [57]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length,
)

target_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length + 1,
  standardize=custom_standardization,
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

#Preparing datasets for the translation task

In [58]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
    "english": eng,
    "sinhala": spa[:, :-1],
    }, spa[:, 1:])
def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape: {inputs['english'].shape}")
  print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
  print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)


In [60]:
print(list(train_ds.as_numpy_iterator())[20])

({'english': array([[  3, 709,   0, ...,   0,   0,   0],
       [  2,  30, 144, ...,   0,   0,   0],
       [175, 815,   0, ...,   0,   0,   0],
       ...,
       [191, 101,   0, ...,   0,   0,   0],
       [  5,  20,   0, ...,   0,   0,   0],
       [ 21,  74,   0, ...,   0,   0,   0]]), 'sinhala': array([[   3,  312, 1198, ...,    0,    0,    0],
       [   2,  127,    0, ...,    0,    0,    0],
       [   2,  161,    0, ...,    0,    0,    0],
       ...,
       [  99,  179,    0, ...,    0,    0,    0],
       [   2,  200,    0, ...,    0,    0,    0],
       [ 170,   21,    0, ...,    0,    0,    0]])}, array([[ 312, 1198,    0, ...,    0,    0,    0],
       [ 127,    0,    0, ...,    0,    0,    0],
       [ 161,    0,    0, ...,    0,    0,    0],
       ...,
       [ 179,    0,    0, ...,    0,    0,    0],
       [ 200,    0,    0, ...,    0,    0,    0],
       [  21,    0,    0, ...,    0,    0,    0]]))


#Transformer encoder implemented as a subclassed layer

In [61]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(
      num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation="relu"),
        layers.Dense(embed_dim),]
    )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()
  def call(self, inputs, mask=None):
    if mask is not None:
        mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
        inputs, inputs, attention_mask=mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
      "embed_dim": self.embed_dim,
      "num_heads": self.num_heads,
      "dense_dim": self.dense_dim,
    })
    return config

#The transformer decorder

In [62]:
class TransformerDecoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention_1 = layers.MultiHeadAttention(
      num_heads=num_heads, key_dim=embed_dim)
    self.attention_2 = layers.MultiHeadAttention(
      num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
      [layers.Dense(dense_dim, activation="relu"),
      layers.Dense(embed_dim),]
    )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()
    self.layernorm_3 = layers.LayerNormalization()
    self.supports_masking = True

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
    })
    return config

  def get_causal_attention_mask(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    i = tf.range(sequence_length)[:, tf.newaxis]
    j = tf.range(sequence_length)
    mask = tf.cast(i >= j, dtype="int32")
    mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1),
        tf.constant([1, 1], dtype=tf.int32)], axis=0)
    return tf.tile(mask, mult)
  def call(self, inputs, encoder_outputs, mask=None):
    causal_mask = self.get_causal_attention_mask(inputs)
    if mask is not None:
      padding_mask = tf.cast(
          mask[:, tf.newaxis, :], dtype="int32")
      padding_mask = tf.minimum(padding_mask, causal_mask)
    else:
      padding_mask = mask
    attention_output_1 = self.attention_1(
        query=inputs,
        value=inputs,
        key=inputs,
        attention_mask=causal_mask)
    attention_output_1 = self.layernorm_1(inputs + attention_output_1)
    attention_output_2 = self.attention_2(
        query=attention_output_1,
        value=encoder_outputs,
        key=encoder_outputs,
        attention_mask=padding_mask,
    )
    attention_output_2 = self.layernorm_2(
        attention_output_1 + attention_output_2)
    proj_output = self.dense_proj(attention_output_2)
    return self.layernorm_3(attention_output_2 + proj_output)

#The Positional Encoding

In [63]:
class PositionalEmbedding(layers.Layer):
  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(
        input_dim=input_dim, output_dim=output_dim)
    self.position_embeddings = layers.Embedding(
      input_dim=sequence_length, output_dim=output_dim)
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions

  def compute_mask(self, inputs, mask=None):
    return tf.math.not_equal(inputs, 0)

  def get_config(self):
    config = super(PositionalEmbedding, self).get_config()
    config.update({
        "output_dim": self.output_dim,
        "sequence_length": self.sequence_length,
        "input_dim": self.input_dim,
    })
    return config

In [64]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [65]:
transformer.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding_4 (Po  (None, None, 256)            3845120   ['english[0][0]']             
 sitionalEmbedding)                                                                               
                                                                                                  
 positional_embedding_5 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']       

#Training the sequence-to-sequence Transformer

In [69]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
39/39 [==============================] - 174s 4s/step - loss: 4.3745 - accuracy: 0.4222 - val_loss: 4.2484 - val_accuracy: 0.4711
Epoch 2/30
39/39 [==============================] - 122s 3s/step - loss: 3.7180 - accuracy: 0.4956 - val_loss: 4.0544 - val_accuracy: 0.4921
Epoch 3/30
39/39 [==============================] - 123s 3s/step - loss: 3.3581 - accuracy: 0.5325 - val_loss: 3.8500 - val_accuracy: 0.5101
Epoch 4/30
39/39 [==============================] - 123s 3s/step - loss: 2.9612 - accuracy: 0.5784 - val_loss: 3.7001 - val_accuracy: 0.5260
Epoch 5/30
39/39 [==============================] - 124s 3s/step - loss: 2.6708 - accuracy: 0.6098 - val_loss: 3.6299 - val_accuracy: 0.5346
Epoch 6/30
39/39 [==============================] - 123s 3s/step - loss: 2.7585 - accuracy: 0.5970 - val_loss: 3.5863 - val_accuracy: 0.5317
Epoch 7/30
39/39 [==============================] - 124s 3s/step - loss: 2.2271 - accuracy: 0.6627 - val_loss: 3.5316 - val_accuracy: 0.5339
Epoch 8/30
39

In [71]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
  tokenized_input_sentence = source_vectorization([input_sentence])
  decoded_sentence ="[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = target_vectorization(
        [decoded_sentence])[:,:-1]
    predictions = transformer(
        [tokenized_input_sentence,tokenized_target_sentence])
    sampled_token_index = np.argmax(predictions[0,i,:])
    sampled_token = spa_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
I saw a dog.
[start] බල්ලෙක් දැක්කා[end]     දැක්කා[end]   දැක්කා[end]          
-
Are you lost?
[start] අතරමං වී තිබේද[end]           කළා[end]      
-
Stay away.
[start] ඉන්න[end]                   
-
I tried.
[start] කළා[end]  කළා[end]       කළා[end]          
-
Are you bald?
[start] තට්ටය[end]         තට්ටය[end]          
-
Go find out.
[start] යන්න[end]                   
-
Keep them.
[start] save කරන්න[end]                  
-
I screamed.
[start] කෑ ගැහුවා[end]        කළා[end]          
-
Look at us.
[start] දිහා බලන්න[end]                  
-
We waited.
[start] බලා සිටියේය[end]                  
-
Come forward.
[start] එන්න[end]                   
-
Hold this.
[start] සුවඳ බලන්න[end]                  
-
I like pizza.
[start] ඒකට කැමතියි[end]        කැමතියි[end]          
-
I like opera.
[start] ඒකට කැමති වුණා[end]                 
-
That's my CD.
[start] මගේ සීඩී එක[end]       ඒක කළා[end]         
-
I talked.
[start] කතා කළා[end]        කළා[end]          
-
We can go.
[start] ය